<a href="https://colab.research.google.com/github/GnomeRapiere/TD-Blockchain-Automating-Trading/blob/main/TD6Blockchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Premières requêtes sur l'API Binance :

In [2]:
import requests as r

request1 = r.get('https://api.binance.com/api/v3/ping')
print(request1.text)

request2 = r.get('https://api.binance.com/api/v3/time')
print(request2.text)

request3 = r.get('https://api.binance.com/api/v3/exchangeInfo')
print(request3.text)

print(request3.json()["symbols"])


{}
{"serverTime":1640948967556}


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Extraction des tokens disponibles :

In [3]:
request4 = r.get('https://api.binance.com/api/v3/ticker/price')
#print(request4.text)
liste = request4.json()
listepaire = []

for el in liste:
  listepaire.append(el["symbol"])

#On vire les doublons si il y en a :

listepaire = list(set(listepaire))
print(listepaire)

#problème ici on a des couples de monnaie et pas les monnaies.
#On récupère ceux-ci sur la requête 3.
listsymb=request3.json()["symbols"]
symboles=[]
for el in listsymb:
  symboles.append(el.get("quoteAsset"))
uniqsymb = list(dict.fromkeys(symboles))
print(uniqsymb)






['CAKEBRL', 'TRXXRP', 'STRAXETH', 'ZILETH', 'BNBUPUSDT', 'ETHDAI', 'POLYBNB', 'TNTETH', 'COMPUSDT', 'CDTBTC', 'USDSUSDT', 'MDABTC', 'LINKETH', 'LINKPAX', 'BEARUSDT', 'WAVESPAX', 'FETUSDT', 'BTTBRL', 'EOSBUSD', 'TNBBTC', 'CELOBTC', 'CMTBTC', 'USDTBRL', 'QNTBNB', 'ONTBUSD', 'WAXPBNB', 'AEETH', 'TRXUPUSDT', 'CITYUSDT', 'JASMYUSDT', 'BEAMBNB', 'BUSDUSDT', 'WABIBNB', 'TRXTRY', 'QUICKUSDT', 'QLCETH', 'NEOPAX', 'ILVBTC', 'LTOBNB', 'LUNAETH', 'GHSTBUSD', 'VETTRY', 'KSMBUSD', 'XMRBUSD', 'LINKEUR', 'CHZTRY', 'MFTETH', 'GNOBTC', 'FTMBRL', 'BCHABCUSDC', 'FRONTETH', 'AIONETH', 'SANDTRY', 'COTIBUSD', 'SKLUSDT', 'SNTETH', 'ZILBNB', 'BLZBNB', 'XLMBNB', 'MATICETH', 'GOBNB', 'SLPUSDT', 'SWRVBNB', 'BEARBUSD', 'IOSTETH', 'GVTETH', 'DASHBNB', 'ENJBUSD', 'LTOBTC', 'XEMBNB', 'ARUSDT', 'ETHUSDT', 'PAXGUSDT', 'LENDBKRW', 'NANOBUSD', 'LITBUSD', 'TUSDBUSD', 'NASBTC', 'USDCBNB', 'ATABTC', 'LSKBNB', 'IOTAUSDT', 'MATICBTC', 'XRPBRL', 'ETHRUB', 'BADGERBTC', 'DASHBUSD', 'OCEANBUSD', 'ACMBUSD', 'BNBBUSD', 'NASETH', 'M

Affichage du ask et du bid :

In [4]:
request5 = r.get('https://api.binance.com/api/v3/ticker/bookTicker')
#print(liste[2])


def getDepth(direction,pair): #A l'appel de la fonction, il faut préciser si on veut le ask ou le bid et préciser la paire qu'on veut.
  liste = request5.json()
  for el in liste:
    if(el["symbol"]==pair):
      #print("Affichage du",direction,":",el[direction])
      return el[direction]

test = getDepth("bidPrice","BTCUSDT")


Order Book (affichage du ask et du bid simultanément)

In [5]:
def orderBook(pair):
  print("Affichage du askPrice:",getDepth("askPrice",pair))
  print("Affichage du bidPrice:",getDepth("bidPrice",pair))
  return (getDepth("askPrice",pair),getDepth("bidPrice",pair))
 
(ask,bid)= orderBook("BTCUSDT")

Affichage du askPrice: 48083.41000000
Affichage du bidPrice: 48083.40000000


Création de la fonction refreshDataCandle()

In [6]:
def refreshDataCandle(pair,duration):
  url="https://api.binance.com/api/v3/klines"
  request6=r.get(url+"?symbol="+pair+"&interval="+duration).json()
  return request6[0]

print(refreshDataCandle("BTCUSDT","5m"))

[1640799000000, '47622.95000000', '47656.75000000', '47522.92000000', '47572.83000000', '122.36194000', 1640799299999, '5821411.17281160', 4309, '52.57677000', '2501813.62634170', '0']


Création de tables SQLite

In [7]:
import sqlite3

conn = sqlite3.connect('candleTab.db')
conn.execute("CREATE TABLE my_table (Id INTEGER PRIMARY KEY autoincrement, date INT, high REAL, low REAL, open REAL, close REAL, volume REAL);")
#conn.execute("CREATE TABLE table2(ld INTEGER PRIMARY KEY autoincrement, uuid TEXT, traded_crypto TEXT, price REAL, created_at_int INT, side TEXT);")
#conn.execute("CREATE TABLE last_checks(ld INTEGER PRIMARY KEY autoincrement, exchange TEXT, trading_pair TEXT, duration TEXT, table_name TEXT, last_check INT, startdate INT, last_id INT);")


Remplissage de la première table (avec high, low, open, close, volume)

In [8]:
def remplissageMyTable():
  index = 0
  for el in listepaire:
    infos = refreshDataCandle(el,"5m") 
    index = index + 1 #on auto incrémente l'index à chaque passage de la boucle pour les id
    conn.execute("INSERT INTO my_table(Id,date,high,low,open,close,volume) VALUES("+str(index)+","+str(infos[0])+","+str(infos[2])+","+str(infos[3])+","+str(infos[1])+","+str(infos[4])+","+str(infos[5])+")")

remplissageMyTable()

Affichage de la table

In [9]:
for row in conn.execute('SELECT DISTINCT * FROM my_table'):
  print(row)


(1, 1640164800000, 69.03, 69.03, 69.03, 69.03, 4.56)
(2, 1640799000000, 0.09311, 0.09284, 0.09297, 0.09288, 45227.9)
(3, 1640799000000, 0.000374, 0.000374, 0.000374, 0.000374, 4.0)
(4, 1640799000000, 2.115e-05, 2.108e-05, 2.115e-05, 2.109e-05, 9182.0)
(5, 1640799000000, 551.34, 546.71, 550.94, 547.62, 7.59)
(6, 1640799000000, 3774.72, 3761.99, 3773.37, 3762.42, 15.3855)
(7, 1585765200000, 0.001512, 0.001512, 0.001512, 0.001512, 0.0)
(8, 1605025200000, 1.133e-05, 1.127e-05, 1.127e-05, 1.133e-05, 115879.0)
(9, 1640799000000, 205.3, 204.3, 205.2, 204.5, 53.102)
(10, 1634379600000, 2.91e-06, 2.88e-06, 2.88e-06, 2.91e-06, 176370.0)
(11, 1592832000000, 0.9979, 0.9979, 0.9979, 0.9979, 0.0)
(12, 1640799000000, 9.68e-06, 9.66e-06, 9.66e-06, 9.66e-06, 2079.0)
(13, 1640799000000, 0.005424, 0.005416, 0.005418, 0.005416, 198.33)
(14, 1589998800000, 4.025, 4.025, 4.025, 4.025, 0.0)
(15, 1585498800000, 13.95, 13.82, 13.92, 13.89, 8794.0377)
(16, 1569680400000, 0.8423, 0.8423, 0.8423, 0.8423, 0.0)
(17

Création d'ordre:

In [14]:
def createOrder(direction, price, amount, pair = 'BTCUSD_d', orderType = 'LimitOrder'):
    data={
        'symbol': pair, 
        'side':direction, 
        'price': price,
        'quoteOrderQty': amount,
        'type': orderType
    }
    request = r.Request('POST','https://api.binance.com/api/v3/order',data=data)

createOrder("ask",200,1000)

Annulation d'ordre :

In [17]:
def cancelOrder(uuid, pair = 'BTCUSD_d'):
    data={
        'symbol': pair, 
        'orderId':uuid
    }

    request = r.Request('DELETE', 'https://api.binance.com/api/v3/order', data=data)

cancelOrder(1)